# Trabalho 4

## Grupo 24

### Pedro Faria - A72640
### Hugo Costeira - A87976

# 1

Para efetuar esta prova podemos construir um FOTS que modela o programa,


       assume m >= 0 and n >= 0 and r == 0 and x == m and y == n 
    0: while y > 0:
    1:    if y & 1 == 1: 
                y , r  = y-1 , r+x
    2:    x , y = x<<1  ,  y>>1
    3: assert r == m * n

In [ ]:
!pip install z3-solver

In [3]:
from z3 import*

def declare(i):
  estado = {}

  estado['x'] = BitVec('x' + str(i),16)
  estado['y'] = BitVec('y' + str(i),16)
  estado['r'] = BitVec('r' + str(i),16)
  estado['m'] = BitVec('m' + str(i),16)
  estado['n'] = BitVec('n' + str(i),16)
  estado['ap'] = Int('ap'+str(i)) # Esta variável será útil para indicar a instrução atual

  return estado


O estado inicial do First Order Transtition System (FOTS) é determinado pelo predicado init. 

Olhando para a pré-condição deste programa, o predicado init corresponde a:

$$ap == 0 \wedge  m \geq 0 \wedge n \geq 0 \wedge r == 0 \wedge x == m \wedge y == n$$

In [4]:
def init(estado):
  l = []

  l.append(estado['m'] >= 0)
  l.append(estado['n'] >= 0)
  l.append(estado['r'] == 0)
  l.append(estado['x'] == estado['m'])
  l.append(estado['y'] == estado['n'])
  l.append(estado['ap'] == 0)

  return And(l)

Em seguida, determinaremos a função de transição. Quando a variável $ap == 0$, estaremos na verificação da condição do ciclo e transitaremos para dentro do ciclo ou para o final do programa (no caso em que a condição $y>0$ é falsa). 

 Estas duas possibilidades traduzem-se em:



  $$ap == 0 \land y > 0 \land y' == y \land m' == m \land r' == r \land x' == x \land n' == n \land ap' == 1$$

   $$ap == 0 \land y \leq 0 \land y' == y \land m' == m \land r' == r \land x' == x \land n' == n \land ap' == 3$$



Resumidamente, sabemos que quando $ap == 1$ entramos no ciclo.

 A próxima instrução é novamente uma condição que terá duas situações: a condição é verdadeira e são alterados os valores das variáveis $y\,,\,r$ ou  a condição é falsa e as variáveis $y\,,\,r$ não são alteradas. 
 
 Estas duas possibilidades traduzem-se em:


 $$ap == 1 \land y\,\&\, 1 == 1 \land y' == y-1 \land m' == m \land r' == r+x \land x' == x \land n' == n \land ap' == 2$$
 $$ap == 1 \land \neg(y\&1 == 1) \land y' == y \land m' == m \land r' == r \land x' == x \land n' == n \land ap' == 2$$



No caso em que $ap==2$ é necessário executar as últimas instruções do ciclo e reiniciar o ap a 0, de modo a que a condição de ciclo seja novamente testada:

$$ap == 2 \land x' == x << 1 \land m' == m \land r' == r \land y' == y >> 1 \land n' == n \land ap' == 0$$

Resta, portanto, adicionar a transição do estado final para ele próprio:
$$ap == 3 \land y' == y \land m' == m \land r' == r \land x' == x \land n' == n \land ap' == 3$$

In [10]:
def trans(curr,prox):
  l = []

  l.append(And(curr['ap'] == 0, curr['y'] > 0, prox['y'] == curr['y'], 
               prox['m'] == curr['m'], prox['r'] == curr['r'], 
               prox['x'] == curr['x'], prox['n'] == curr['n'], prox['ap'] == 1))
  
  l.append(And(curr['ap'] == 0, curr['y'] <= 0, prox['y'] == curr['y'], 
               prox['m'] == curr['m'], prox['r'] == curr['r'], 
               prox['x'] == curr['x'], prox['n'] == curr['n'], prox['ap'] == 3))
  
  l. append(And(curr['ap'] == 1, curr['y'] & 1 == 1, prox['y'] == curr['y'], 
                prox['r'] == curr['r'] + curr['x'], prox['x'] == curr['x'], 
                prox['m'] == curr['m'], prox['n'] == curr['n'], prox['ap'] == 2))
  
  l. append(And(curr['ap'] == 1, Not(curr['y'] & 1 == 1), prox['y'] == curr['y'], 
                prox['r'] == curr['r'], prox['x'] == curr['x'], 
                prox['m'] == curr['m'], prox['n'] == curr['n'], prox['ap'] == 2))
  
  l. append(And(curr['ap'] == 2, prox['x'] == curr['x'] << 1, 
                prox['y'] == curr['y'] >> 1, prox['m'] == curr['m'], 
                prox['n'] == curr['n'], prox['r'] == curr['r'], prox['ap'] == 0))
  
  l.append(And(curr['ap'] == 3, prox['y'] == curr['y'], 
               prox['m'] == curr['m'], prox['r'] == curr['r'], 
               prox['x'] == curr['x'], prox['n'] == curr['n'], prox['ap'] == 3))
  
  return Or(l)

Seguidamente, apresenta-se uma função que gera um traço da execução do FOTS construído:

In [14]:
def gera_traco(declare,init,trans,k):
  trace = [declare(i) for i in range(k)]
  s = Solver()

  s.add(init(trace[0]))

  for i in range(k-1):
    s.add(trans(trace[i],trace[i+1]))

  r = s.check()
  if r == sat:
    m = s.model()
    for i in range(k):
      print("estado: "+str(i))
      for v in trace[i]:
        print(v, '=', m[trace[i][v]])
    return
  
  print('UNSAT')
  return

gera_traco(declare,init,trans,20)

estado: 0
x = 16
y = 57
r = 0
m = 16
n = 57
ap = 0
estado: 1
x = 16
y = 57
r = 0
m = 16
n = 57
ap = 1
estado: 2
x = 16
y = 57
r = 16
m = 16
n = 57
ap = 2
estado: 3
x = 32
y = 28
r = 16
m = 16
n = 57
ap = 0
estado: 4
x = 32
y = 28
r = 16
m = 16
n = 57
ap = 1
estado: 5
x = 32
y = 28
r = 16
m = 16
n = 57
ap = 2
estado: 6
x = 64
y = 14
r = 16
m = 16
n = 57
ap = 0
estado: 7
x = 64
y = 14
r = 16
m = 16
n = 57
ap = 1
estado: 8
x = 64
y = 14
r = 16
m = 16
n = 57
ap = 2
estado: 9
x = 128
y = 7
r = 16
m = 16
n = 57
ap = 0
estado: 10
x = 128
y = 7
r = 16
m = 16
n = 57
ap = 1
estado: 11
x = 128
y = 7
r = 144
m = 16
n = 57
ap = 2
estado: 12
x = 256
y = 3
r = 144
m = 16
n = 57
ap = 0
estado: 13
x = 256
y = 3
r = 144
m = 16
n = 57
ap = 1
estado: 14
x = 256
y = 3
r = 400
m = 16
n = 57
ap = 2
estado: 15
x = 512
y = 1
r = 400
m = 16
n = 57
ap = 0
estado: 16
x = 512
y = 1
r = 400
m = 16
n = 57
ap = 1
estado: 17
x = 512
y = 1
r = 912
m = 16
n = 57
ap = 2
estado: 18
x = 1024
y = 0
r = 912
m = 16
n = 57
ap 

# 2